The goal in this file is comparing the solution of regular Bounded-Makespan-Min-Cost with Fixed_x_Bounded_Makespan_Min_Cost and generate the figure automatically.

In [1]:
# importing packages
import numpy as np
import pandas as pd
from gurobipy import *


from bokeh.io import output_notebook, output_file, curdoc, push_notebook, show
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, Slider, ColumnDataSource, CustomJS
from bokeh.layouts import row, column, gridplot, widgetbox
from bokeh.models.widgets import Panel, Tabs, Button

from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

import numpy as np
import pandas as pd

from gurobipy import *

output_notebook()

Loading BokehJS ...

In [2]:
scnCount = 30
subScnCount = 2

Number = [2]
SubNumber = [1]
directory = "./SSScnBased_S332_" 

In [3]:
# reading data
def readData(readNumber, readSubNumber):  
        jobs = 30
        testNumber = readNumber # 1 to 48
        subTestNumber = readSubNumber # 1 to 10
        fileName = 'J' + str(jobs) + str(testNumber) + '_' + str(subTestNumber) + '.RCP'
        file = '../dataRCPSP/A' + str(jobs) + '/j' + str(jobs) + 'rcp/' + fileName
        my_cols = ["0", "1", "2", "3", "4", "5", "6", "7", "8"]
        df = pd.read_table(file, sep='\t',delim_whitespace=True, engine='python', names=my_cols)
        duration = df.iloc[2:, 0].values
        resourcesUsage = df.iloc[2:, 1:5].values
        successorsCount = df.iloc[2:, 5].values
        successors = df.iloc[2:, 6:9].values
        resourcesCount = int(df.iloc[0, 1])
        jobsCount = int(df.iloc[0,0])
        availableResources = df.iloc[1,0:resourcesCount].values
        return duration, resourcesUsage, successors, resourcesCount, jobsCount, availableResources

In [4]:
# model sets and parameters for FCT model
def FCTSetParam(jobsCount, resourcesCount, duration, resourcesUsage, availableResources, scenarioCount, subScenarioCount):  
        activities = np.array(range(jobsCount))
        resources = np.array(range(resourcesCount))
        scenarios = np.array(range(scenarioCount))
        subScenarios = np.array(range(subScenarioCount))
        T = duration.sum()
        d = dict(zip(activities,resourcesUsage))
        M = 150
        N = availableResources.max()
        return activities, resources, d, T, M , N, scenarios, subScenarios

In [5]:
# scenario generating
def scenarioGen(scenarios, jobsCount, duration, activities):
    np.random.seed(0)
    rndAct = np.random.rand(jobsCount)
    durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    for s in scenarios:
        for a in activities:
            cr = np.random.rand()
            if (cr >= 0.5) : durationScenarios[a,s] = duration[a] * (1 + 3/22 * rndAct[a])
            elif (cr < 0.5) : durationScenarios[a,s] = duration[a] * (1 - 3/22 * rndAct[a])
    p = dict(zip(activities,durationScenarios))
    return p

In [6]:
# define variables of FCT model
def addFCTVars(activities, resources, T, availableResources):  
        x = FCT.addVars(activities, activities, lb=0.0, ub=1.0, vtype='B', 
                       name="X") # one if activity i complete before activity j starts
        s = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        o = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="O") # due date of procurements for activity i
        e = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="E") # earliness of materials for activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return x, s, f, o, e

In [7]:
# define variables of FCT model
def addFCTVarsMM(activities, resources, T, availableResources):  
        x = FCT.addVars(activities, activities, lb=0.0, ub=1.0, vtype='B', 
                       name="X") # one if activity i complete before activity j starts
        s = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return x, s, f

In [8]:
# define variables of FCT model
def addFCTVarsXX(activities, resources, T, availableResources):  
        s = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        o = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="O") # due date of procurements for activity i
        e = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="E") # earliness of materials for activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return s, f, o, e

In [9]:
# constraint 2.5 of FCT model
def addFCTConst1(activities, successors, x):    
        FCT.addConstrs(
            (x[activityI, activityJ] 
             ==
             1
             for activityI in activities if activityI != activities[-1]
             for activityJ in activities[successors[activityI][successors[activityI] > 0].astype(np.int) - 1]), 
            name = "NetworkRelations")

In [10]:
# constraint 2.6a of FCT model
def addFCTConst2a(activities, x, s, p, M, scenarios):  
        FCT.addConstrs(
            (s[activityJ, scenario] -  s[activityI, scenario] 
             >=
             p[activityI][scenario] - M * (1 - x[activityI, activityJ])
             for scenario in scenarios
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]), 
            name = "NetworkStartTimeRelations")

In [11]:
# constraint 2.6a of FCT model
def addFCTConst2aMM(activities, x, s, ppp, M):  
        FCT.addConstrs(
            (s[activityJ] -  s[activityI] 
             >=
             ppp[activityI] - M * (1 - x[activityI, activityJ])
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]), 
            name = "NetworkStartTimeRelations")

In [12]:
# constraint 2.6b of FCT model
def addFCTConst2b(activities, s, o, scenarios):  
        FCT.addConstrs(
            (s[activity, scenario] 
             >=
             o[activity]
             for scenario in scenarios
             for activity in activities), 
            name = "NetworkStartTimeAndOrderRelations")

In [13]:
# constraint 2.7 of FCT model
def addFCTConst3(activities, x, f, resources, N):  
        FCT.addConstrs(
            (f[activityI, activityJ, resource] -  N * x[activityI, activityJ] 
             <=
             0
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]
             for resource in resources), 
            name = "NetworkFlowRelations")

In [14]:
# constraint 2.8 of FCT model
def addFCTConst4(activities, f, resources, d):  
        FCT.addConstrs(
            (quicksum(f[activityI, activityJ, resource] 
                      for activityJ in activities 
                      if activityJ != activities[0]) 
             ==
             d[activityI][resource]
             for activityI in activities if activityI != activities[0] 
             for resource in resources), 
            name = "OutgoingFlows")

In [15]:
# constraint 2.9 of FCT model
def addFCTConst5(activities, f, resources, d):  
        FCT.addConstrs(
            (quicksum(f[activityI, activityJ, resource] 
                      for activityI in activities 
                      if activityI != activities[-1]) 
             ==
             d[activityJ][resource]
             for activityJ in activities if activityJ != activities[-1] 
             for resource in resources), 
            name = "IngoingFlows")

In [16]:
# constraint 2.10a of FCT model
def addFCTConst6(activities, f, resources, availableResources):  
        FCT.addConstrs(
            (quicksum(f[activities[0], activityJ, resource] 
                      for activityJ in activities 
                      if activityJ != activities[0]) 
             ==
             availableResources[resource]
             for resource in resources), 
            name = "FirstFlow")

In [17]:
# constraint 2.10b of FCT model
def addFCTConst7(activities, f, resources, availableResources):  
        FCT.addConstrs(
            (quicksum(f[activityI, activities[-1], resource] 
                      for activityI in activities 
                      if activityI != activities[-1]) 
             ==
             availableResources[resource]
             for resource in resources), 
            name = "LastFlow")

In [18]:
# constraint for bounding makespan
def addFCTConst9(activities, scenarios, scenarioCount, s, makespan):  
        FCT.addConstrs(
            (quicksum(s[activities[-1], scenario] for scenario in scenarios)
             <=
             makespan * scenarioCount
             for alaki in range(0,1)
             ), 
            name = "Makespandddd")

In [19]:
# Objective of FCT model -- min cost
def addFCTObj(activities, s, scenarios, scenarioCount, o): 
        obj = (quicksum(s[activity, scenario] - o[activity] 
                              for activity in activities
                              for scenario in scenarios) / scenarioCount)
        FCT.setObjective(obj, GRB.MINIMIZE)

In [20]:
# Objective of FCT model -- min makespan det
def addFCTObjMM(activities, s):
  
        obj = (s[activities[-1]])
        FCT.setObjective(obj, GRB.MINIMIZE)

In [21]:
# Objective of FCT model -- min expected makespan
def addFCTObjdW(activities, s, scenarios, scenarioCount):
  
        obj = (quicksum(s[activities[-1], scenario] for scenario in scenarios) / scenarioCount)
        FCT.setObjective(obj, GRB.MINIMIZE)

In [22]:
for i in Number:
    for j in SubNumber: 

        
#MM##########################################################################################################
################################# MIN makespan for DETERMINISTIC
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
        print("    MIN makespan for DETERMINISTIC    step=1  ")
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
############################################################################################################1

        makespanDet = 0
        makespanExp = 0
             
        duration, resourcesUsage, successors, resourcesCount, jobsCount, availableResources = readData(i, j)
        activities, resources, d, T, M , N, scenarios, subScenarios = \
            FCTSetParam(jobsCount, resourcesCount, duration, resourcesUsage, availableResources, scnCount, subScnCount)
        p = scenarioGen(scenarios, jobsCount, duration, activities)
        pMM = list(range(jobsCount))
        for act in range(jobsCount):
            pMM[act] = sum(p[act]) / scnCount
        FCT = Model('FCT: MIN makespan for DETERMINISTIC')
        #FCT.setParam('TimeLimit', 3600)
        x, s, f = addFCTVarsMM(activities, resources, T, availableResources)
        addFCTConst1(activities, successors, x)
        addFCTConst2aMM(activities, x, s, pMM, M)
        addFCTConst3(activities, x, f, resources, N)
        addFCTConst4(activities, f, resources, d)
        addFCTConst5(activities, f, resources, d)
        addFCTConst6(activities, f, resources, availableResources)
        addFCTConst7(activities, f, resources, availableResources)
        FCT.update()
        addFCTObjMM(activities, s)
        FCT.optimize()        

#.............................................. Save MM-X

        directory = directory + str(i) + "_" + str(j) + "/"
        !mkdir $directory
        if FCT.Status == 2:

            # Open a file X
            xoName = directory+ "MM-x.txt"
            xo = open(xoName, "a")
            for v in range(jobsCount):
                if v != 0 : xo.write("\n")
                for k in range(jobsCount):
                    xo.write(str(int(x[v, k].X)) + "\t")
            # Close opend file X
            xo.close()
                     
#.............................................. Update eff.txt

        preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|." + str(FCT.Status) + ".|MM|\t\t"
        if FCT.Status == 2:
            # Open a file
            do = open(directory + "eff.txt", "a")
            do.write(preName)
            do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
            # Close opend file
            do.close()
        else:
            # Open a file
            do = open(directory + "eff.txt", "a")
            do.write(preName)
            do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
            # Close opend file
            do.close()       
        
        
#dW############################################################################################################
##################### MIN E(makespan) for SCENARIO base | XdW bede
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
        print("    MIN E(makespan) for SCENARIO base | XdW bede   step=2 ")
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
##############################################################################################################2

        subP = scenarioGen(subScenarios, jobsCount, duration, activities)
        FCT = Model('FCT: MIN makespan for SCENARIO base')
        #FCT.setParam('TimeLimit', 3600)
        x, s, f, o, e = addFCTVars(activities, resources, T, availableResources)
        addFCTConst1(activities, successors, x)
        addFCTConst2a(activities, x, s, subP, M, subScenarios)
        addFCTConst2b(activities, s, o, subScenarios)
        addFCTConst3(activities, x, f, resources, N)
        addFCTConst4(activities, f, resources, d)
        addFCTConst5(activities, f, resources, d)
        addFCTConst6(activities, f, resources, availableResources)
        addFCTConst7(activities, f, resources, availableResources)
        FCT.update()
        addFCTObjdW(activities, s, subScenarios, subScnCount)
        FCT.optimize()        

#.............................................. Save dW-X

        if FCT.Status == 2:
            # Open a file X
            xoName = directory + "dW-x.txt"
            xo = open(xoName, "a")
            for v in range(jobsCount):
                if v != 0 : xo.write("\n")
                for k in range(jobsCount):
                    xo.write(str(int(x[v, k].X)) + "\t")
            # Close opend file X
            xo.close()
            
#.............................................. Update eff.txt

        # Calculations
        if FCT.MIPGap <= 10000:
            aaa = 0 # Expected Makespan
            for vvv in range(subScnCount):
                aaa += s[jobsCount - 1, vvv].X
        preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|." + str(FCT.Status) + ".|dW|\t\t"
        if FCT.Status == 2:
            # Open a file
            do = open(directory + "eff.txt", "a")
            do.write(preName)
            do.write(str(round(aaa / subScnCount,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
            # Close opend file
            do.close()
        else: 
            # Open a file
            do = open(directory + "eff.txt", "a")
            do.write(preName)
            do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
            # Close opend file
            do.close()                 
                        
#SdW###############################################################################################################
####################################   for fixed X: MIN E(makespan) | use x of dW shema bede
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
        print("  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3 ")
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
#################################################################################################################3

        fileName =  "dW-x.txt"
        file = directory + fileName
        x = np.loadtxt(file)
        x = x.astype(int)

        FCT = Model('for fixed X: MIN scenario used makespan | use x of dW')
        #FCT.setParam('TimeLimit', 10*60)
        s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
#             addFCTConst1(activities, successors, x)
        addFCTConst2a(activities, x, s, p, M, scenarios)
        addFCTConst2b(activities, s, o, scenarios)
        addFCTConst3(activities, x, f, resources, N)
        addFCTConst4(activities, f, resources, d)
        addFCTConst5(activities, f, resources, d)
        addFCTConst6(activities, f, resources, availableResources)
        addFCTConst7(activities, f, resources, availableResources)
#         addFCTConst9(activities, scenarios, scnCount, s, makespan)
        FCT.update()
        addFCTObjdW(activities, s, scenarios, scnCount)
        FCT.optimize()
        
#.............................................. Update eff.txt

        # Calculations
        preName = "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|." + str(FCT.Status) + ".|SdW|\t\t"
        if FCT.Status == 2:
            # Open a file
            makespanExp = FCT.ObjVal
            do = open(directory + "eff.txt", "a")
            do.write(preName)
            do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
            # Close opend file
            do.close()
        else:  
            # Open a file
            do = open(directory + "eff.txt", "a")
            do.write(preName)
            do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
            # Close opend file
            do.close() 
            

            
#SMM###############################################################################################################
####################################   for fixed X: MIN E(makespan) | use x of MM shema bede
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
        print("  for fixed X: MIN E(makespan) | use x of MM shema bede   step=4 ")
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
#################################################################################################################4

        fileName =   "MM-x.txt"
        file = directory + fileName
        x = np.loadtxt(file)
        x = x.astype(int)

        FCT = Model('for fixed X: MIN scenario used makespan | use x of MM')
        #FCT.setParam('TimeLimit', 10*60)
        s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
#             addFCTConst1(activities, successors, x)
        addFCTConst2a(activities, x, s, p, M, scenarios)
        addFCTConst2b(activities, s, o, scenarios)
        addFCTConst3(activities, x, f, resources, N)
        addFCTConst4(activities, f, resources, d)
        addFCTConst5(activities, f, resources, d)
        addFCTConst6(activities, f, resources, availableResources)
        addFCTConst7(activities, f, resources, availableResources)
#         addFCTConst9(activities, scenarios, scnCount, s, makespan)
        FCT.update()
        addFCTObjdW(activities, s, scenarios, scnCount)
        FCT.optimize()
        
#.............................................. Update eff.txt

        # Calculations
        preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|." + str(FCT.Status) + ".|SMM|\t\t"
        if FCT.Status == 2:
            # Open a file
            makespanDet = FCT.ObjVal
            do = open(directory + "eff.txt", "a")
            do.write(preName)
            do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
            # Close opend file
            do.close()
        else:  
            # Open a file
            do = open(directory + "eff.txt", "a")
            do.write(preName)
            do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
            # Close opend file
            do.close() 
            

#XX##########################################################################################################
######################   for fixed X, bounded makespan: MIN cost | use x of MM
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
        print("  for fixed X, bounded makespan: MIN cost | use x of MM   step=5_0 ")
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
############################################################################################################5

        fileName =   "MM-x.txt"
        file = directory + fileName
        x = np.loadtxt(file)
        x = x.astype(int)
        
        makespansXX = makespanDet * np.array([1, 1.001, 1.002, 1.003, 1.005, 1.0075, 1.01, 1.015, 1.02, 1.025, 1.03, 1.05, 1.1, 1.2])
        XX = {}
        shomarande = 0
        for makespan in makespansXX:
            shomarande += 1
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
            print("  for fixed X, bounded makespan: MIN cost | use x of MM   step=5: " + str(shomarande) + "/" + str(len(makespansXX)))
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
        
            FCT = Model('FCT: for fixed X, bounded makespan: MIN cost')
            #FCT.setParam('TimeLimit', 10*60)
            s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
#             addFCTConst1(activities, successors, x)
            addFCTConst2a(activities, x, s, p, M, scenarios)
            addFCTConst2b(activities, s, o, scenarios)
            addFCTConst3(activities, x, f, resources, N)
            addFCTConst4(activities, f, resources, d)
            addFCTConst5(activities, f, resources, d)
            addFCTConst6(activities, f, resources, availableResources)
            addFCTConst7(activities, f, resources, availableResources)
            addFCTConst9(activities, scenarios, scnCount, s, makespan)
            FCT.update()
            addFCTObj(activities, s, scenarios, scnCount, o)
            FCT.optimize()
            
#.............................................. Update eff.txt
            
            # Calculations
            preName =   "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|." + str(FCT.Status) + ".|XX|\t\t"
            if FCT.Status == 2:
                ccc = 0 # Worst Makespan
                for vvv in range(scnCount):
                    if ccc < s[jobsCount - 1, vvv].X:
                        ccc = s[jobsCount - 1, vvv].X
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write(str(round(makespan,4)) + "\t\t" + str(round(FCT.ObjVal,4)) + "   \t" + str(round(ccc,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                XX[makespan] = FCT.ObjVal
                # Close opend file
                do.close()
            else:  
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close() 
                
                
#WW##############################################################################################################
###########################   for fixed X, bounded makespan: MIN cost | use x of dW
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
        print("  for fixed X, bounded makespan: MIN cost | use x of dW   step=6_0 ")
        print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
################################################################################################################6

        fileName =   "dW-x.txt"
        file = directory + fileName
        x = np.loadtxt(file)
        x = x.astype(int)
        
        mianji1 = 1 + (makespanDet-makespanExp)/(makespanExp) * 0.2
        mianji2 = 1 + (makespanDet-makespanExp)/(makespanExp) * 0.4
        mianji3 = 1 + (makespanDet-makespanExp)/(makespanExp) * 0.5
        mianji4 = 1 + (makespanDet-makespanExp)/(makespanExp) * 0.6
        mianji5 = 1 + (makespanDet-makespanExp)/(makespanExp) * 0.7
        mianji6 = 1 + (makespanDet-makespanExp)/(makespanExp) * 0.8
        mianji7 = 1 + (makespanDet-makespanExp)/(makespanExp) * 0.9

        makespansWW = np.hstack((makespanExp * np.array([1, mianji1, mianji2, mianji3, mianji4, mianji5, mianji6, mianji7]), makespansXX))
        WW = {}
        shomarande = 0
        for makespan in makespansWW:
            
            shomarande += 1
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
            print("  for fixed X, bounded makespan: MIN cost | use x of dW   step=6: " + str(shomarande) + "/" + str(len(makespansWW)))
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
        
            FCT = Model('FCT: for fixed X, bounded makespan: MIN cost')
            #FCT.setParam('TimeLimit', 10*60)
            s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
#             addFCTConst1(activities, successors, x)
            addFCTConst2a(activities, x, s, p, M, scenarios)
            addFCTConst2b(activities, s, o, scenarios)
            addFCTConst3(activities, x, f, resources, N)
            addFCTConst4(activities, f, resources, d)
            addFCTConst5(activities, f, resources, d)
            addFCTConst6(activities, f, resources, availableResources)
            addFCTConst7(activities, f, resources, availableResources)
            addFCTConst9(activities, scenarios, scnCount, s, makespan)
            FCT.update()
            addFCTObj(activities, s, scenarios, scnCount, o)
            FCT.optimize()

#.............................................. Update eff.txt

            # Calculations
            preName =  "A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|." + str(FCT.Status) + ".|WW|\t\t"
            if FCT.Status == 2:
                ccc = 0 # Worst Makespan
                for vvv in range(scnCount):
                    if ccc < s[jobsCount - 1, vvv].X:
                        ccc = s[jobsCount - 1, vvv].X
                WW[makespan] = FCT.ObjVal
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write(str(round(makespan,4)) + "\t\t" + str(round(FCT.ObjVal,4)) + "   \t" + str(round(ccc,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()
            else:  
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close() 
                
                
#FIGURE##########################################################################################################
############################################ FIGURE
################################################################################################################7

        vWW = np.array(list(WW.values()))
        kWW = np.array(list(WW.keys()))
        dfWW = pd.DataFrame({'kWW':kWW, 'vWW':vWW})
        sourceWW = ColumnDataSource(dfWW)

        vXX = np.array(list(XX.values()))
        kXX = np.array(list(XX.keys()))
        dfXX = pd.DataFrame({'kXX':kXX, 'vXX':vXX})
        sourceXX = ColumnDataSource(dfXX)

        # plotting

        plt = figure(plot_width=1200, plot_height=800, title="Efficient Frontier", tools='pan,wheel_zoom', x_axis_label='Expected Completion Time', y_axis_label='Expected Inventory Cost')

        plt.line(x='kWW', y='vWW', source=sourceWW, line_width=10, alpha=.3, color='red')
        plt.circle(x='kWW', y='vWW', source=sourceWW, line_width=10, alpha=.3, legend="Heuristic", color='red', hover_fill_color='firebrick', hover_alpha=1,
                  hover_line_color='white')

        plt.line(x='kXX', y='vXX', source=sourceXX, line_width=10, alpha=.3, color='black')
        plt.circle(x='kXX', y='vXX', source=sourceXX, line_width=10, alpha=.3, legend="Deterministic", color='black', hover_fill_color='firebrick', hover_alpha=1,
                  hover_line_color='white')

        plt.legend.location='top_right'
        plt.legend.background_fill_color='floralwhite'

        hover = HoverTool(tooltips=None)
        plt.add_tools(hover)

        pltName = "ScnBased_" + str(i) + "_" + str(j) + ".html"
        output_file(pltName)
        show(plt)

        curdoc().clear()

iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_2_1
    MIN makespan for DETERMINISTIC    step=1  
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Optimize a model with 5109 rows, 5152 columns and 18495 nonzeros
Variable types: 4128 continuous, 1024 integer (1024 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 2e+02]
Presolve removed 4574 rows and 4654 columns
Presolve time: 0.01s
Presolved: 535 rows, 498 columns, 1660 nonzeros
Variable types: 286 continuous, 212 integer (212 binary)

Root relaxation: objective 3.367567e+01, 247 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   33.67567    0    5          -   33.67567      -     -    0s
     0     0   33.67567    0    6          -   33.67567      -     -    0s
     0     0   33.67567

 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.19031803e+02  5.38922790e+01  1.70e+00 4.93e+00  4.69e+00     0s
   1  -4.06386026e+01  1.32564871e+02  2.74e-01 5.05e+00  8.51e-01     0s
   2   4.54732064e+01  1.31215601e+02  2.36e-02 5.73e-01  1.04e-01     0s
   3   3.03026928e+01  5.92794070e+01  9.84e-04 3.51e-02  1.67e-02     0s
   4   3.39500071e+01  4.00374669e+01  8.33e-17 4.79e-03  3.28e-03     0s
   5   3.71580101e+01  3.87400375e+01  7.91e-16 1.36e-03  8.55e-04     0s
   6   3.81301428e+01  3.83690015e+01  2.08e-16 3.92e-04  1.30e-04     0s
   7   3.82462673e+01  3.82700737e+01  2.64e-16 4.45e-05  1.31e-05     0s

Barrier performed 7 iterations in 0.10 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with primal simplex
Solved in 54 iterations and 0.13 seconds
Optimal objective  3.825325456e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_2_1


  for fixed X, bounded makespan: MIN cost | use x of MM   step=5: 3/14
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Optimize a model with 33891 rows, 6048 columns and 69590 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e+00]
  Bounds range     [2e+01, 1e+02]
  RHS range        [9e-01, 1e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 5077 rows and 37489 columns
Presolve time: 0.06s
Presolved: 971 rows, 2450 columns, 4972 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.060e+03
 Factor NZ  : 2.056e+04 (roughly 2 MBytes of memory)
 Factor Ops : 5.439e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.79463213e+03  3.20692400e+01  3.66e+00 3.33e+00  5.00e+01     0s
   1  -1.41826117e+03  6.30412952e+02  1.76


Iteration    Objective       Primal Inf.    Dual Inf.      Time
     125    1.1489320e+01   0.000000e+00   0.000000e+00      0s
     125    1.1489320e+01   0.000000e+00   0.000000e+00      0s

Solved with barrier
Solved in 125 iterations and 0.18 seconds
Optimal objective  1.148931954e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_2_1
  for fixed X, bounded makespan: MIN cost | use x of MM   step=5: 6/14
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Optimize a model with 33891 rows, 6048 columns and 69590 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e+00]
  Bounds range     [2e+01, 1e+02]
  RHS range        [9e-01, 1e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 5053 rows and 36674 columns
Presolve time: 0.07s
Presolved: 995 rows, 3265 columns, 6602 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.875e+03
 Factor NZ  : 2.517e+04 (roughl

   0   4.29294385e+03  2.99553403e+01  5.96e+00 6.59e+00  3.55e+01     0s
   1   5.66369688e+02  7.22183005e+02  1.14e+00 8.79e+01  1.14e+01     0s
   2  -2.04230366e+03  5.72211902e+02  4.63e-05 1.70e+01  2.74e+00     0s
   3  -1.00044069e+03  1.83871302e+02  4.53e-06 1.48e+00  4.00e-01     0s
   4  -5.80725863e+02  5.41795421e+01  2.23e-06 2.54e-01  1.57e-01     0s
   5  -1.32424999e+02  2.85565331e+01  4.26e-07 1.01e-01  3.90e-02     0s
   6  -5.16020746e+01  1.66553283e+01  1.66e-07 3.60e-02  1.60e-02     0s
   7  -1.94187561e+01  9.36006411e+00  6.93e-08 9.38e-03  6.57e-03     0s
   8  -3.82013228e+00  6.68621556e+00  2.43e-08 2.84e-03  2.38e-03     0s
   9   4.05744045e+00  5.59299342e+00  3.10e-09 4.02e-04  3.48e-04     0s
  10   4.97344727e+00  5.36953543e+00  8.01e-10 1.05e-04  8.88e-05     0s
  11   5.30449829e+00  5.30723735e+00  4.23e-15 6.07e-08  5.99e-07     0s
  12   5.30641882e+00  5.30642166e+00  2.62e-15 6.95e-12  6.19e-10     0s
  13   5.30642074e+00  5.30642074e+00 

  for fixed X, bounded makespan: MIN cost | use x of MM   step=5: 13/14
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Optimize a model with 33891 rows, 6048 columns and 69590 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e+00]
  Bounds range     [2e+01, 1e+02]
  RHS range        [9e-01, 1e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 5053 rows and 34816 columns
Presolve time: 0.05s
Presolved: 995 rows, 5123 columns, 10287 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 5.702e+03
 Factor NZ  : 4.268e+04 (roughly 3 MBytes of memory)
 Factor Ops : 2.982e+06 (less than 1 second per iteration)
 Threads    : 18

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.47449017e+03 -0.00000000e+00  7.30e+00 1.11e+01  4.58e+01     0s
   1  -2.80589250e+02  1.16671460e+03  1


Solved with barrier
Solved in 0 iterations and 0.06 seconds
Infeasible model
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_2_1
  for fixed X, bounded makespan: MIN cost | use x of dW   step=6: 4/22
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Optimize a model with 33891 rows, 6048 columns and 69590 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e+00]
  Bounds range     [2e+01, 1e+02]
  RHS range        [9e-01, 1e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 4935 rows and 9758 columns
Presolve time: 0.03s

Solved with barrier
Solved in 0 iterations and 0.03 seconds
Infeasible model
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_2_1
  for fixed X, bounded makespan: MIN cost | use x of dW   step=6: 5/22
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Optimize a model with 33891 rows, 6048 columns and 69590 nonzeros
Coefficient statistics:
  Matrix range     [

  11   1.75529207e+01  1.76702523e+01  6.83e-14 3.74e-05  5.71e-05     0s
  12   1.76289373e+01  1.76357641e+01  4.99e-14 4.80e-07  3.02e-06     0s
  13   1.76329920e+01  1.76330001e+01  5.04e-14 7.50e-10  3.62e-09     0s
  14   1.76329966e+01  1.76329966e+01  4.50e-14 2.22e-13  4.28e-15     0s

Barrier solved model in 14 iterations and 0.10 seconds
Optimal objective 1.76329966e+01

Crossover log...

       0 DPushes remaining with DInf 1.5631940e-13                 0s

      69 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase complete: Pinf 0.0000000e+00, Dinf 1.5276669e-13      0s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
      94    1.7632997e+01   0.000000e+00   0.000000e+00      0s

Solved with primal simplex
Solved in 1231 iterations and 0.12 seconds
Optimal objective  1.763299659e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_2_1
  for fixed X, bounded makesp

  14   8.63717202e+00  8.63717613e+00  4.56e-14 2.31e-10  9.96e-10     0s
  15   8.63717517e+00  8.63717518e+00  4.10e-13 7.37e-14  1.07e-12     0s

Barrier solved model in 15 iterations and 0.11 seconds
Optimal objective 8.63717517e+00

Crossover log...

       0 DPushes remaining with DInf 5.8619776e-14                 0s

     106 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase complete: Pinf 0.0000000e+00, Dinf 7.1054274e-14      0s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     131    8.6371752e+00   0.000000e+00   0.000000e+00      0s
     131    8.6371752e+00   0.000000e+00   0.000000e+00      0s

Solved with barrier
Solved in 131 iterations and 0.17 seconds
Optimal objective  8.637175175e+00
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_2_1
  for fixed X, bounded makespan: MIN cost | use x of dW   step=6: 18/22
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii

   2  -3.04983517e+03  6.64305126e+02  3.99e-04 1.90e+01  3.00e+00     0s
   3  -1.39622492e+03  2.10885955e+02  2.79e-05 1.32e+00  4.30e-01     0s
   4  -1.77436282e+02  4.63454492e+01  8.91e-07 9.25e-02  4.75e-02     0s
   5  -5.84320969e+01  1.88258735e+01  3.00e-07 2.06e-02  1.56e-02     0s
   6  -3.19880842e+01  1.04908411e+01  1.66e-07 8.50e-03  8.47e-03     0s
   7  -1.68338690e+01  5.36726814e+00  8.65e-08 3.67e-03  4.40e-03     0s
   8  -1.12018561e+01  3.25431484e+00  5.59e-08 2.15e-03  2.86e-03     0s
   9  -3.80917330e+00  1.56298286e+00  1.70e-08 1.06e-03  1.06e-03     0s
  10  -1.40545995e+00  3.59711897e-01  5.99e-09 1.55e-04  3.48e-04     0s
  11  -8.51169343e-02  1.75839409e-02  2.85e-10 2.52e-06  2.01e-05     0s
  12  -1.73314717e-04  3.39912045e-05  5.38e-13 3.80e-09  4.06e-08     0s
  13  -1.73315838e-07  3.79247496e-08  9.02e-16 4.08e-13  4.13e-11     0s
  14  -5.15593316e-13  7.81597009e-13  5.53e-16 4.55e-13  1.95e-16     0s

Barrier solved model in 14 iterations